In [1]:
# Résumé des données manquantes par sujet et bundle (avec confounds spécifiques DEP et actimétrie)
import os
import pandas as pd
from os.path import join as opj
from actiDep.set_config import get_HCP_bundle_names
from actiDep.data.loader import Actidep
# Paramètres
DB_ROOT = '/home/ndecaux/NAS_EMPENN/share/projects/actidep/bids'
PIPELINE = 'hcp_association_50pts'  # adapter si besoin
METRIC_COLS = ['FA','MD','RD','AD','IFW','IRF']
CLASSIF_VARS = ['group','apathy']
CORR_VARS = ['ami','aes']
MANDATORY_VARS = list(set(['age','sex','group']))
DEP_VARS = list(set(MANDATORY_VARS + CLASSIF_VARS + CORR_VARS))
POINT_COL_CANDIDATES = ['point','point_id']
ACTIMETRY_XLSX = opj(DB_ROOT,'actimetry_features.xlsx')

ds = Actidep(db_root=DB_ROOT)
all_subjects= ds.subject_ids

csv_files = ds.get_global(pipeline=PIPELINE, extension='csv',datatype='metric')
present_subjects = list(set([f.subject for f in csv_files]))


all_bundles=list(get_HCP_bundle_names().keys())
bundles_in_db = list(set([f.get_entities().get('bundle',None) for f in csv_files if f.bundle in all_bundles]))
print("Sujets totaux dans la BDD :", len(all_subjects))
print("Sujets avec données dans le pipeline :", len(present_subjects))
print("Sujets sans données dans le pipeline :", set(all_subjects)-set(present_subjects))
print(len(csv_files), "fichiers CSV trouvés")

print("Nombre de bundles dans la BDD :", len(bundles_in_db))
print("Nombre de bundles attendus :", len(all_bundles))

Sujets totaux dans la BDD : 61
Sujets avec données dans le pipeline : 60
Sujets sans données dans le pipeline : {'03026'}
4260 fichiers CSV trouvés
Nombre de bundles dans la BDD : 71
Nombre de bundles attendus : 71


In [2]:
for sub in present_subjects:
    sub_files = [f for f in csv_files if f.subject==sub]
    bundles_for_sub = list(set([f.get_entities().get('bundle',None) for f in sub_files if f.bundle in all_bundles]))
    missing_bundles = list(set(all_bundles)-set(bundles_for_sub))
    if len(missing_bundles)>0:
        print(f"Sujet {sub} : {len(missing_bundles)} bundles manquants :", missing_bundles)
    else:
        print(f"Sujet {sub} : tous les bundles sont présents ({len(bundles_for_sub)})")

Sujet 03015 : tous les bundles sont présents (71)
Sujet 03011 : tous les bundles sont présents (71)
Sujet 01040 : tous les bundles sont présents (71)
Sujet 01001 : tous les bundles sont présents (71)
Sujet 01021 : tous les bundles sont présents (71)
Sujet 01016 : tous les bundles sont présents (71)
Sujet 01010 : tous les bundles sont présents (71)
Sujet 03022 : tous les bundles sont présents (71)
Sujet 01008 : tous les bundles sont présents (71)
Sujet 03009 : tous les bundles sont présents (71)
Sujet 03024 : tous les bundles sont présents (71)
Sujet 01026 : tous les bundles sont présents (71)
Sujet 01039 : tous les bundles sont présents (71)
Sujet 03005 : tous les bundles sont présents (71)
Sujet 01038 : tous les bundles sont présents (71)
Sujet 01029 : tous les bundles sont présents (71)
Sujet 01032 : tous les bundles sont présents (71)
Sujet 03016 : tous les bundles sont présents (71)
Sujet 01018 : tous les bundles sont présents (71)
Sujet 03018 : tous les bundles sont présents (71)


### Infos participants manquantes

In [4]:
from pprint import pprint

participant_file = os.path.join(DB_ROOT,'participants_full_info.xlsx')
participants_df = pd.read_excel(participant_file)
print("Informations disponibles dans participants_full_info.xlsx :", participants_df.columns.tolist())
participants_df['subject_id']=participants_df['participant_id'].str.replace('sub-','')
participants_df.set_index('subject_id',inplace=True)
subjects_df = pd.DataFrame(present_subjects,columns=['subject_id'])
participants_df#MANDATORY_VARS]
subjects_df = subjects_df.merge(participants_df, left_on='subject_id', right_on='subject_id', how='left')

mandatory = subjects_df[['subject_id']+MANDATORY_VARS]
#Get lines that contain NaN
mandatory = mandatory[mandatory.isnull().any(axis=1)]

deps = subjects_df[['subject_id']+DEP_VARS][subjects_df['group'].isin(['dep',''])]
#Get lines that contain NaN
deps = deps[deps.isnull().any(axis=1)]
deps
print("Données manquantes pour les variables communes (dep et controles) :", len(mandatory))
pprint(mandatory)

print("Données manquantes pour les variables spécifiques DEP :", len(deps))
pprint(deps)


Informations disponibles dans participants_full_info.xlsx : ['participant_id', 'initial', 'city', 'group', 'age', 'sex', 'nse', 'atcd_endoc', 'updrs', 'matthis', 'type_dep', 'duration_dep', 'cgi', 'madrs', 'apathy', 'fatigue', 'aes', 'ami_ba', 'ami_sm', 'ami_es', 'fluency', 'fluency_s', 'stroop', 'tmt_a', 'tmt_ba', 'mcst_cat', 'mcst_error', 'mcst_pers', 'ami', 'ami_manual', 'acp1_scores_cliniques', 'pc_act1', 'pc_act2', 'pc_act3', 'pc_act4', 'subject', 'shanoir', 'antidep', 'antipsycho', 'deviation', 'atcd_cv', 'atcd_neuro', 'atcd_rhumato', 'atcd_pneumo']
Données manquantes pour les variables communes (dep et controles) : 0
Empty DataFrame
Columns: [subject_id, age, group, sex]
Index: []
Données manquantes pour les variables spécifiques DEP : 1
   subject_id  age  apathy   aes  ami group sex
56      03010   78     2.0  45.0  NaN   dep   f


In [5]:
acti_df=pd.read_excel(ACTIMETRY_XLSX)
acti_df['subject_id']=acti_df['participant_id'].str.replace('sub-','')
acti_df.set_index('subject_id',inplace=True)
missing_acti = subjects_df.merge(acti_df, left_on='subject_id', right_on='subject_id', how='left')
acti_columns = acti_df.columns.tolist()
acti_columns.remove('participant_id')
missing_acti = missing_acti[['subject_id','group','apathy']+acti_columns]

#List lines that contain NaN
missing_acti = missing_acti[missing_acti[acti_columns].isnull().any(axis=1)]
missing_acti

,subject_id,group,apathy,inactivity_mean_3d,inactivity_std_3d,inactivity_min_3d,inactivity_max_3d,activity_mean_3d,activity_std_3d,activity_min_3d,...,oadl_fft_min_12h_3,oadl_fft_min_12h_4,oadl_fft_min_12h_5,oadl_fft_min_12h_6,oadl_fft_max_12h_1,oadl_fft_max_12h_2,oadl_fft_max_12h_3,oadl_fft_max_12h_4,oadl_fft_max_12h_5,oadl_fft_max_12h_6
11,01026,dep,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,03025,hc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,01035,dep,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,01006,dep,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,01037,hc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,01044,dep,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
wrong_bundles= []
for sub in present_subjects:
    sub_files = [f for f in csv_files if f.subject==sub]
    for bundle_file in sub_files:
        
        bundle = bundle_file.get_entities().get('bundle',None)
        metric_df= pd.read_csv(bundle_file.path)
        missing_points=metric_df[metric_df['FA'].isnull()]
        missing_points['subject_id']=sub
        missing_points['bundle']=bundle
        wrong_bundles.append(missing_points)


wrong_bundles = pd.concat(wrong_bundles)


/tmp/ipykernel_793299/2951286078.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_points['subject_id']=sub
/tmp/ipykernel_793299/2951286078.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_points['bundle']=bundle
/tmp/ipykernel_793299/2951286078.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [7]:
wrong_bundles_by_sub = wrong_bundles.groupby(['subject_id']).size().reset_index(name='n_missing_points')
wrong_bundles_by_sub_and_bundle = wrong_bundles.groupby(['subject_id','bundle']).size().reset_index(name='n_missing_points')
# wrong_bundles_by_bundle = wrong_bundles.groupby(['bundle']).size().reset_index(name='n_missing_points')
# wrong_bundles_by_bundle_and_point = wrong_bundles.groupby(['bundle','point_id']).size().reset_index(name='n_missing_points')

# Merge group and apathy columns only for dataframes that have subject_id
dataframes_with_subject = {
    'wrong_bundles_by_sub': wrong_bundles_by_sub,
    'wrong_bundles_by_sub_and_bundle': wrong_bundles_by_sub_and_bundle
}

for df_name, df in dataframes_with_subject.items():
    if 'subject_id' in df.columns:
        df = df.merge(subjects_df[['subject_id', 'group', 'apathy']], on='subject_id', how='left')
        globals()[df_name] = df
# Display the updated dataframe as an example
print(wrong_bundles_by_sub.head())
print(wrong_bundles_by_sub_and_bundle.head())

wrong_bundles_by_bundle = wrong_bundles_by_sub_and_bundle.groupby(['bundle']).size().reset_index(name='n_missing_points')

wrong_bundles_by_sub_and_bundle['ratio_dep']=wrong_bundles_by_sub_and_bundle['subject_id'].map(lambda x: 1. if subjects_df[subjects_df['subject_id']==x]['group'].values[0]=='dep' else 0.)

# wrong_bundles_by_sub_and_bundle.groupby('bundle')['ratio_dep'].mean()
wrong_bundles_by_bundle['ratio_dep']=wrong_bundles_by_sub_and_bundle.groupby('bundle')['ratio_dep'].mean().to_list()

wrong_bundles_by_sub_and_bundle['ratio_apathy']=wrong_bundles_by_sub_and_bundle['subject_id'].map(lambda x: 1. if subjects_df[subjects_df['subject_id']==x]['apathy'].values[0]==2 else (0. if subjects_df[subjects_df['subject_id']==x]['apathy'].values[0]==1 else None))
wrong_bundles_by_bundle['ratio_apathy']=wrong_bundles_by_sub_and_bundle.groupby('bundle')['ratio_apathy'].mean().to_list()
wrong_bundles_by_bundle

  subject_id  n_missing_points group  apathy
0      01001                16    hc     NaN
1      01002                13   dep     2.0
2      01006                27   dep     2.0
3      01007                21   dep     1.0
4      01008                96    hc     NaN
  subject_id      bundle  n_missing_points group  apathy
0      01001      CGleft                 1    hc     NaN
1      01001    SCPright                 2    hc     NaN
2      01001  SLFIIright                13    hc     NaN
3      01002     AFright                 3   dep     2.0
4      01002          CA                 1   dep     2.0


,bundle,n_missing_points,ratio_dep,ratio_apathy
0,AFleft,13,0.615385,0.875000
1,AFright,26,0.730769,0.789474
2,ATRleft,1,1.000000,1.000000
3,ATRright,2,1.000000,1.000000
4,CA,30,0.600000,0.555556
5,CC1,1,1.000000,1.000000
6,CC4,1,1.000000,1.000000
7,CC5,1,1.000000,1.000000
8,CC7,1,1.000000,1.000000
9,CGleft,17,0.705882,0.833333


In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import os, pandas as pd, datetime as dt, matplotlib.pyplot as plt, seaborn as sns
from os.path import join as opj
report_dir = opj(DB_ROOT,'quality_reports')
os.makedirs(report_dir, exist_ok=True)

print("Bundle avec points manquants :" , len(wrong_bundles_by_bundle_and_point['bundle'].unique()))
unique_bundles = wrong_bundles_by_bundle_and_point['bundle'].unique()
num_bundles = len(unique_bundles)
fig, axes = plt.subplots(5, 7, figsize=(20, 15), constrained_layout=True)

for i, bundle in enumerate(unique_bundles):
    row, col = divmod(i, 7)
    ax = axes[row, col]
    data = wrong_bundles_by_bundle_and_point[wrong_bundles_by_bundle_and_point['bundle'] == bundle].fillna({'point_id': 0})
    
    sns.barplot(data=data, x='point_id', y='n_missing_points', ax=ax, native_scale=True)
    ax.set_title(f'{bundle}')
    ax.set_xlabel('Point ID')
    ax.set_ylabel('Missing Points')
    ax.set_xlim(0, 50)

# Hide any unused subplots
for j in range(num_bundles, 5 * 7):
    row, col = divmod(j, 7)
    fig.delaxes(axes[row, col])

plt.suptitle('Missing subject points for each bundle', fontsize=16)
fig_path = opj(report_dir,'missing_points_by_bundle.png')
plt.savefig(fig_path)
    
    


    
    

NameError: name 'wrong_bundles_by_bundle_and_point' is not defined

# Rapport synthétique des données manquantes et complétude

Ce rapport est généré dynamiquement :

- Couverture bundles par sujet

- Variables mandatoires / classification / corrélation / confondants (spécifique DEP)

- Actimétrie : présence des colonnes et valeurs manquantes

- Points de tractométrie manquants par bundle et sujet

- Export CSV détaillés + un récap global

Exécuter la cellule suivante pour (re)générer tous les fichiers.

In [ ]:
# Rapport consolidé (version complète)

timestamp = dt.datetime.now().isoformat(timespec='seconds')

# Couverture bundles par sujet (recalcule simple)
coverage_rows = []
for sub in sorted(present_subjects):
    sub_files = [f for f in csv_files if f.subject==sub]
    bundles_for_sub = set([f.get_entities().get('bundle',None) for f in sub_files if f.bundle in all_bundles])
    missing = set(all_bundles)-bundles_for_sub
    coverage_rows.append({'subject':sub,'n_present':len(bundles_for_sub),'n_missing':len(missing)})
coverage_df = pd.DataFrame(coverage_rows)
coverage_df.to_csv(opj(report_dir,'coverage_bundles_per_subject.csv'), index=False)

# Export des tables existantes (complètes)
if 'mandatory' in globals():
    mandatory.to_csv(opj(report_dir,'mandatory_rows_with_missing.csv'), index=False)
if 'deps' in globals():
    deps.to_csv(opj(report_dir,'dep_rows_with_missing.csv'), index=False)
if 'missing_acti' in globals():
    missing_acti.to_csv(opj(report_dir,'actimetry_rows_with_missing.csv'), index=False)

if 'wrong_bundles' in globals():
    wrong_bundles.to_csv(opj(report_dir,'missing_points_raw.csv'), index=False)
    wrong_bundles_by_sub_and_bundle.to_csv(opj(report_dir,'missing_points_by_subject_bundle.csv'), index=False)
    wrong_bundles_by_bundle_and_point.to_csv(opj(report_dir,'missing_points_by_bundle_point.csv'), index=False)

# Préparer résumés spécifiques pour wrong_bundles*
if 'wrong_bundles' in globals():
    top_subjects = (wrong_bundles_by_sub.sort_values('n_missing_points', ascending=False)
                                      .head(20))
    top_bundles = (wrong_bundles_by_bundle.sort_values('n_missing_points', ascending=False)
                                      .head(20))
else:
    top_subjects = pd.DataFrame()
    top_bundles = pd.DataFrame()

from tabulate import tabulate

# Construction du markdown avec tabulate
sections = []
sections.append('# Rapport Qualité Données Tractométrie')
sections.append(f'Généré: {timestamp}')
sections.append('')
sections.append('## 1. Couverture bundles par sujet (table complète)')
sections.append(tabulate(coverage_df, headers='keys', tablefmt='pipe', showindex=False))
sections.append('')

if 'mandatory' in globals() and not mandatory.empty:
    sections.append('## 2. Lignes sujets avec variables mandatoires manquantes (table complète)')
    sections.append(tabulate(mandatory, headers='keys', tablefmt='pipe', showindex=False))

if 'deps' in globals() and not deps.empty:
    sections.append('## 3. Lignes sujets DEP avec variables DEP manquantes (table complète)')
    sections.append(tabulate(deps, headers='keys', tablefmt='pipe', showindex=False))

if 'missing_acti' in globals() and not missing_acti.empty:
    sections.append('## 4. Sujets avec au moins une valeur actimétrie manquante')
    sections.append(tabulate(pd.DataFrame({'subject_id': missing_acti['subject_id'].unique()}), headers='keys', tablefmt='pipe', showindex=False))

if not top_subjects.empty:
    sections.append('## 5. Sujets avec le plus de points manquants (Top 20)')
    sections.append(tabulate(top_subjects, headers='keys', tablefmt='pipe', showindex=False))
if not top_bundles.empty:
    sections.append('## 6. Bundles avec le plus de points manquants (Top 20)')
    sections.append(tabulate(top_bundles, headers='keys', tablefmt='pipe', showindex=False))

if fig_path:
    sections.append('## 7. Figure : Points manquants par bundle')
    sections.append(f'![Missing subject points for each bundle]({os.path.basename(fig_path)})')

report_md_path = opj(report_dir, 'quality_report_full.md')
with open(report_md_path, 'w') as f:
    f.write('\n\n'.join(sections))
print('Rapport markdown écrit:', report_md_path)
print('Fichiers disponibles dans', report_dir)


NameError: name 'dt' is not defined

In [27]:
import subprocess
try:
    pdf_path = opj(report_dir, 'quality_report_full.pdf')
    subprocess.run(['quarto', 'render', report_md_path, '--to', 'typst'], check=True)
    print('Rapport PDF généré:', pdf_path)
except Exception as e:
    print('Erreur lors de la génération du PDF:', e)
    print('Assurez-vous que Quarto est installé et accessible dans votre environnement.')
# Fin du script

pandoc 
  to: typst
  output-file: quality_report_full.typ
  standalone: true
  default-image-extension: svg
  wrap: none
  citeproc: false
  variables: {}
  
[typst]: Compiling quality_report_full.typ to quality_report_full.pdf...[typst]: Compiling quality_report_full.typ to quality_report_full.pdf...

Rapport PDF généré: /home/ndecaux/NAS_EMPENN/share/projects/actidep/bids/quality_reports/quality_report_full.pdf


DONE

Output created: quality_report_full.pdf

